In [108]:
#%pip install -U scikit-learn
# Carga Bibliotecas e Dados
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

def idade_media(df_train: pd.DataFrame, df_test: pd.DataFrame):
    _df_test = pd.DataFrame.copy(df_test)
    _df_train = pd.DataFrame.copy(df_train)

    _df_test['Survived'] = 0

    df = pd.concat([_df_train, _df_test], ignore_index=True)
    df.dropna(subset=['Age'], how='all', inplace=True)
    return df.groupby('Saudacao')['Age'].mean()

def set_idade_media_if_null(row: pd.Series, media):
    result = row['Age']
    if ( np.isnan(row['Age']) ):
        result = media[row['Saudacao']]
    return result

def tipoIdade(x):
    if x < 18:
        return 'Crianca'
    elif x >= 18 and x < 50:
        return 'Adulto'
    else:
        return 'Idoso'



In [109]:
df = pd.read_csv("train.csv")
train, test = train_test_split(df, test_size=0.3, random_state=100)

In [110]:
print(train['Age'].isna().sum())
print(train.shape[0])

print(test['Age'].isna().sum())
print(test.shape[0])

print(train['Sex'].isnull().sum())
print(train.shape[0])

print(test['Sex'].isnull().sum())
print(test.shape[0])

128
623
49
268
0
623
0
268


In [111]:
mod = DecisionTreeClassifier()

train['Tam_Familia'] = train['SibSp'] + train['Parch']
test['Tam_Familia'] = test['SibSp'] + test['Parch']

train['Saudacao'] = train['Name'].str.extract("([A-Za-z]+)\.")
test['Saudacao'] = test['Name'].str.extract("([A-Za-z]+)\.")

_idade_media = idade_media(train, test)
train['Age'] = train.apply(set_idade_media_if_null, axis=1, media=_idade_media)
test['Age'] = test.apply(set_idade_media_if_null, axis=1, media=_idade_media)

#print('Train Age Count: ', train['Age'].isna().sum())
#print('Test Age Count: ', test['Age'].isna().sum())

# Reduzindo possibilidades para padronizar modelo, pois em teste não há todos os tipos existentes e treino
#train['Saudacao']\
#  .replace(['Mlle',     'Mme',      'Ms',       'Dr',    'Major', 'Lady',   'Countess','Jonkheer','Col',  'Rev',  'Capt',  'Sir',   'Don'],
#           ['Senhorita','Senhorita','Senhorita','Senhor','Senhor','Senhora','Senhora', 'Outro',   'Outro','Outro','Senhor','Senhor','Senhor'], inplace=True)
#test['Saudacao']\
#  .replace(['Dona',     'Mlle',     'Mme',      'Ms',       'Dr',    'Major', 'Lady',   'Countess','Jonkheer','Col',  'Rev',  'Capt',  'Sir',   'Don'],
#           ['Senhorita','Senhorita','Senhorita','Senhorita','Senhor','Senhor','Senhora','Senhora', 'Outro',   'Outro','Outro','Senhor','Senhor','Senhor'], inplace=True)

# Reduzindo possibilidades para padronizar modelo, pois em teste não há todos os tipos existentes e treino
train['Saudacao']\
  .replace(['Mlle','Mme', 'Ms',  'Dr','Major','Lady','Countess','Jonkheer','Col',  'Rev',  'Capt','Sir','Don'],
           ['Miss','Miss','Miss','Mr','Mr',   'Mrs', 'Mrs',     'Other',   'Other','Other','Mr',  'Mr', 'Mr'], inplace=True)
test['Saudacao']\
  .replace(['Dona','Mlle','Mme', 'Ms',  'Dr','Major','Lady','Countess','Jonkheer','Col',  'Rev',  'Capt','Sir','Don'],
           ['Miss','Miss','Miss','Miss','Mr','Mr',   'Mrs', 'Mrs',     'Other',   'Other','Other','Mr',  'Mr',  'Mr'], inplace=True)

# Classificando idade por Criança, Adulto, Idoso
train['t_idade'] = train['Age'].apply(lambda x: tipoIdade(x))
test['t_idade'] = test['Age'].apply(lambda x: tipoIdade(x))

In [112]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Tam_Familia,Saudacao,t_idade
69,70,0,3,"Kink, Mr. Vincenz",male,26.000000,2,0,315151,8.6625,NaN,S,2,Mr,Adulto
85,86,1,3,"Backstrom, Mrs. Karl Alfred (Maria Mathilda Gu...",female,33.000000,3,0,3101278,15.8500,NaN,S,3,Mrs,Adulto
794,795,0,3,"Dantcheff, Mr. Ristiu",male,25.000000,0,0,349203,7.8958,NaN,S,0,Mr,Adulto
161,162,1,2,"Watt, Mrs. James (Elizabeth ""Bessie"" Inglis Mi...",female,40.000000,0,0,C.A. 33595,15.7500,NaN,S,0,Mrs,Adulto
815,816,0,1,"Fry, Mr. Richard",male,32.368090,0,0,112058,0.0000,B102,S,0,Mr,Adulto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,856,1,3,"Aks, Mrs. Sam (Leah Rosen)",female,18.000000,0,1,392091,9.3500,NaN,S,1,Mrs,Adulto
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.000000,1,1,11751,52.5542,D35,S,2,Mrs,Adulto
835,836,1,1,"Compton, Miss. Sara Rebecca",female,39.000000,1,1,PC 17756,83.1583,E49,C,2,Miss,Adulto
792,793,0,3,"Sage, Miss. Stella Anna",female,21.773973,8,2,CA. 2343,69.5500,NaN,S,10,Miss,Adulto


In [113]:
test

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Tam_Familia,Saudacao,t_idade
205,206,0,3,"Strom, Miss. Telma Matilda",female,2.00000,0,1,347054,10.4625,G6,S,1,Miss,Crianca
44,45,1,3,"Devaney, Miss. Margaret Delia",female,19.00000,0,0,330958,7.8792,NaN,Q,0,Miss,Adulto
821,822,1,3,"Lulic, Mr. Nikola",male,27.00000,0,0,315098,8.6625,NaN,S,0,Mr,Adulto
458,459,1,2,"Toomey, Miss. Ellen",female,50.00000,0,0,F.C.C. 13531,10.5000,NaN,S,0,Miss,Idoso
795,796,0,2,"Otter, Mr. Richard",male,39.00000,0,0,28213,13.0000,NaN,S,0,Mr,Adulto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,112,0,3,"Zabour, Miss. Hileni",female,14.50000,1,0,2665,14.4542,NaN,C,1,Miss,Crianca
730,731,1,1,"Allen, Miss. Elisabeth Walton",female,29.00000,0,0,24160,211.3375,B5,S,0,Miss,Adulto
105,106,0,3,"Mionoff, Mr. Stoytcho",male,28.00000,0,0,349207,7.8958,NaN,S,0,Mr,Adulto
479,480,1,3,"Hirvonen, Miss. Hildur E",female,2.00000,0,1,3101298,12.2875,NaN,S,1,Miss,Crianca


In [114]:
train.boxplot(column=['Pclass', 'Tam_Familia'])


<Axes: >

In [115]:
test.boxplot(column=['Pclass', 'Tam_Familia'])

<Axes: >

In [116]:
train_embarked_dummies = pd.get_dummies(train.loc[:,['Embarked']])
train_sex_dummies = pd.get_dummies(train.loc[:,['Sex']])
train_saudacao_dummies = pd.get_dummies(train.loc[:,['Saudacao']])
train_t_idade = pd.get_dummies(train.loc[:,['t_idade']])

test_embarked_dummies = pd.get_dummies(test.loc[:,['Embarked']])
test_sex_dummies = pd.get_dummies(test.loc[:,['Sex']])
test_saudacao_dummies = pd.get_dummies(test.loc[:,['Saudacao']])
test_t_idade = pd.get_dummies(test.loc[:,['t_idade']])

In [117]:
X = pd.concat([train.loc[:,[ 'Pclass', 'Tam_Familia']], \
                      train_saudacao_dummies, \
                      train_embarked_dummies,\
                      train_sex_dummies,\
                      train_t_idade\
                        ], axis=1)

X

,Pclass,Tam_Familia,Saudacao_Master,Saudacao_Miss,Saudacao_Mr,Saudacao_Mrs,Saudacao_Other,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,t_idade_Adulto,t_idade_Crianca,t_idade_Idoso
69,3,2,False,False,True,False,False,False,False,True,False,True,True,False,False
85,3,3,False,False,False,True,False,False,False,True,True,False,True,False,False
794,3,0,False,False,True,False,False,False,False,True,False,True,True,False,False
161,2,0,False,False,False,True,False,False,False,True,True,False,True,False,False
815,1,0,False,False,True,False,False,False,False,True,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,3,1,False,False,False,True,False,False,False,True,True,False,True,False,False
871,1,2,False,False,False,True,False,False,False,True,True,False,True,False,False
835,1,2,False,True,False,False,False,True,False,False,True,False,True,False,False
792,3,10,False,True,False,False,False,False,False,True,True,False,True,False,False


In [118]:
# DICA DE PYTHON (muitos perguntam): Se você desejar UNIR dois dataframes utilize o comando=> X = pd.concat([X, X], axis=1) <= no exemplo "dentro" dos sinais > e <, o dataframe X é "duplicado" lado a lado. Faça o teste

 # X = Features / Y = Label
Y = train.loc[:,['Survived']]
mod.fit(X ,Y)

print(mod.score(X, Y))

0.8635634028892456


In [119]:
X_test = pd.concat([test.loc[:,[ 'Pclass', 'Tam_Familia']], \
                      test_saudacao_dummies, \
                      test_embarked_dummies,\
                      test_sex_dummies,\
                      test_t_idade\
                        ], axis=1)

X_test['t_idade_Idoso'] = False

p = mod.predict(X_test)

In [120]:
from sklearn import metrics as M
# As linhas abaixo só montam o arquivo de saída que será utilizado no Kaggle, com destaque para
# como o resultado da variável p é um número entre 0 e 1, convertemos ele para 0 se for menor que 0.5 e para 1 se maior
Survived_Predict = [1 if  i > 0.6 else 0 for i in p]
df_pred = test.loc[:,['PassengerId']]
df_pred['Survived'] = Survived_Predict

print(M.r2_score(test['Survived'], Survived_Predict))
print(M.mean_squared_error(test['Survived'], Survived_Predict))
print(M.accuracy_score(test['Survived'], Survived_Predict))

0.22681899486469337
0.1865671641791045
0.8134328358208955


In [121]:
df_pred.to_csv('predict_M3.csv', index=False) 